In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler


# Load the dataset
data = pd.read_csv("Bank Customer Churn Prediction.csv")

# Display the first few rows and summary info
data.head(30)

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,15574012,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,15592531,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,15656148,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,15792365,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,15592389,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [2]:
# Data Cleaning
# Checking for duplicates and missing values

# Remove duplicates
data_cleaned = data.drop_duplicates()

# Check for missing values in the dataset
missing_values = data_cleaned.isnull().sum()

# Data Transformation
# Adding new features: categorize age into bins and create income brackets for better segmentation

# Define age categories
age_bins = [18, 30, 45, 60, 80]  # Arbitrary age bins
age_labels = ["18-30", "31-45", "46-60", "60+"]
data_cleaned['age_group'] = pd.cut(data_cleaned['age'], bins=age_bins, labels=age_labels, right=False)

# Define income brackets
income_bins = [0, 50000, 100000, 150000, 200000]
income_labels = ["Low", "Medium", "High", "Very High"]
data_cleaned['income_bracket'] = pd.cut(data_cleaned['estimated_salary'], bins=income_bins, labels=income_labels, right=False)

# Display the processed data and any missing values found
data_cleaned.head(), missing_values



(   customer_id  credit_score country  gender  age  tenure    balance  \
 0     15634602           619  France  Female   42       2       0.00   
 1     15647311           608   Spain  Female   41       1   83807.86   
 2     15619304           502  France  Female   42       8  159660.80   
 3     15701354           699  France  Female   39       1       0.00   
 4     15737888           850   Spain  Female   43       2  125510.82   
 
    products_number  credit_card  active_member  estimated_salary  churn  \
 0                1            1              1         101348.88      1   
 1                1            0              1         112542.58      0   
 2                3            1              0         113931.57      1   
 3                2            0              0          93826.63      0   
 4                1            1              1          79084.10      0   
 
   age_group income_bracket  
 0     31-45           High  
 1     31-45           High  
 2     31-45

In [3]:
# Applying comprehensive data cleaning to the dataset

# Starting with the cleaned data from the previous steps
data_cleaned_full = data_cleaned.copy()

# Dropping unnecessary column (customer_id) if present

# Handle Outliers
# Cap extreme values at the 1st and 99th percentiles for key financial fields (credit_score, balance, estimated_salary)
outlier_columns = ['credit_score', 'balance', 'estimated_salary', 'products_number']
# for col in outlier_columns:
#     lower_cap = data_cleaned_full[col].quantile(0.01)
#     upper_cap = data_cleaned_full[col].quantile(0.99)
#     data_cleaned_full[col] = data_cleaned_full[col].clip(lower=lower_cap, upper=upper_cap)

# Fill missing values - Using median for numerical columns, as it is robust to outliers
for col in outlier_columns:
    data_cleaned_full[col].fillna(data_cleaned_full[col].median(), inplace=True)



# Removing duplicates if any exist
data_cleaned_full.drop_duplicates(inplace=True)
# Transformation de la colonne 'gender'
data_cleaned_full['gender'] = data_cleaned_full['gender'].replace({'Female': 'F', 'Male': 'H'})

# Transformation de la colonne 'country' (garder seulement la première lettre)
data_cleaned_full['country'] = data_cleaned_full['country'].str[:2]

# Vérification des changements
data_cleaned_full[['gender', 'country']].head()


# Final check on the cleaned dataset
data_cleaned_full.info(), data_cleaned_full.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   customer_id       10000 non-null  int64   
 1   credit_score      10000 non-null  int64   
 2   country           10000 non-null  object  
 3   gender            10000 non-null  object  
 4   age               10000 non-null  int64   
 5   tenure            10000 non-null  int64   
 6   balance           10000 non-null  float64 
 7   products_number   10000 non-null  int64   
 8   credit_card       10000 non-null  int64   
 9   active_member     10000 non-null  int64   
 10  estimated_salary  10000 non-null  float64 
 11  churn             10000 non-null  int64   
 12  age_group         9985 non-null   category
 13  income_bracket    10000 non-null  category
dtypes: category(2), float64(2), int64(8), object(2)
memory usage: 1.0+ MB


(None,
    customer_id  credit_score country gender  age  tenure    balance  \
 0     15634602           619      Fr      F   42       2       0.00   
 1     15647311           608      Sp      F   41       1   83807.86   
 2     15619304           502      Fr      F   42       8  159660.80   
 3     15701354           699      Fr      F   39       1       0.00   
 4     15737888           850      Sp      F   43       2  125510.82   
 
    products_number  credit_card  active_member  estimated_salary  churn  \
 0                1            1              1         101348.88      1   
 1                1            0              1         112542.58      0   
 2                3            1              0         113931.57      1   
 3                2            0              0          93826.63      0   
 4                1            1              1          79084.10      0   
 
   age_group income_bracket  
 0     31-45           High  
 1     31-45           High  
 2     31-4